In [1]:
import cv2
import numpy as np

In [2]:
# 算法的核心是利用局部窗口在图像上进行移动，判断灰度是否发生较大的变化。
# 如果窗口内的灰度值（在梯度图上）都有较大的变化，那么这个窗口所在区域就存在角点。

https://www.cnblogs.com/ronny/p/4009425.html

![image.png](角点检测示意图.png)


![img](../data/ex007.jpg)

增大
α
的值，将减小角点响应值
R
，降低角点检测的灵性，减少被检测角点的数量；减小
α
值，将增大角点响应值
R
，增加角点检测的灵敏性，增加被检测角点的数量。

In [20]:
# cornerHarris(src, blockSize(窗口大小), ksize(sobel卷积核用于求梯度), k(响应值计算系数0.04)[, dst[, borderType]]) -> dst
img = cv2.imread('../data/ex007.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
dst = cv2.cornerHarris(gray,10,3,0.04)

print(gray.shape,dst.shape,dst.dtype) # 每一个像素都可以计算出一个角点响应值R float32

# 设定阈值,R大于阈值的才认为是角点
# 实际上这一步通过局部非极大值抑制来处理
mask = dst >(0.01*dst.max())
img[mask] = [0,0,255]
cv2.imshow('img',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

(1198, 1200) (1198, 1200) float32


In [16]:
# 角点响应图的展示
min_value = dst.min()
max_value = dst.max()
print(min_value,max_value)
res = (dst-min_value)/(max_value-min_value)*255
res = res.astype(np.uint8)

cv2.imshow('res',res)
cv2.waitKey(0)
cv2.destroyAllWindows()


-0.0018249477 0.009837534


In [19]:
# 自己实现harris算法

blockSize = 5 # 窗口大小
sigma = 1
ksize = 3 # 计算梯度核大小

def get_gaussian_kernel(size, sigma):
    ax = np.linspace(-(size - 1) / 2., (size - 1) / 2., size)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sigma))
    return kernel / np.sum(kernel)

img = cv2.imread('../data/ex007.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# _,gray = cv2.threshold(gray,200,255,cv2.THRESH_BINARY)
# cv2.imshow('img',gray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

gx = cv2.Sobel(gray,-1,dx =1,dy=0)
gy = cv2.Sobel(gray,-1,dx =0,dy=1)

gaussian_kernel = get_gaussian_kernel(blockSize,sigma)
print(gaussian_kernel)

#  M矩阵  [ Ix**2  Ix*Iy ]   A = Ix**2  B = Iy**2  C = IxIy  [ A  C ]
#           Ix*Iy  Iy**2                                       C  B

A = gx**2
B = gy**2
C = gx*gy

# 对一个窗口内的值进行高斯平滑
A = cv2.filter2D(A,-1,gaussian_kernel)
B = cv2.filter2D(B,-1,gaussian_kernel)
C = cv2.filter2D(C,-1,gaussian_kernel)

R = A*B-C**2 - 0.04 *((A+B)**2)

print(R.shape)
print(R)
mask = R >(0.01*R.max())
print(0.01*R.max())

img[mask] = [0,0,255]

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


[[0.00296902 0.01330621 0.02193823 0.01330621 0.00296902]
 [0.01330621 0.0596343  0.09832033 0.0596343  0.01330621]
 [0.02193823 0.09832033 0.16210282 0.09832033 0.02193823]
 [0.01330621 0.0596343  0.09832033 0.0596343  0.01330621]
 [0.00296902 0.01330621 0.02193823 0.01330621 0.00296902]]
(1198, 1200)
[[ 0.0000e+00 -4.0000e-02 -1.6000e-01 ... -4.0000e-02 -4.0000e-02
   0.0000e+00]
 [-4.0000e-02 -1.6000e-01  1.3600e+00 ...  2.3600e+00  8.4000e-01
  -4.0000e-02]
 [-1.6000e-01  6.4000e-01  8.4400e+00 ...  1.1440e+01  3.3600e+00
   1.6400e+00]
 ...
 [ 7.8600e+01  1.8236e+02  2.7360e+01 ...  1.4944e+02  3.9800e+01
   2.4680e+01]
 [ 3.6160e+01  7.3280e+01  1.3648e+02 ...  8.4800e+01  2.2240e+01
   1.2160e+01]
 [ 2.0000e+01  2.5240e+01  5.6600e+01 ...  4.8000e+01  1.3440e+01
   6.5600e+00]]
2.5500000000000003


In [33]:
# shi-tomasi角点检测
#cv2.goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance
#                       [, corners[, mask[, blockSize[, useHarrisDetector[, k]]]]]) -> corners

img = cv2.imread('../data/ex007.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

corners = cv2.goodFeaturesToTrack(gray,1000,0.1,1)
for point in corners:
    x,y = point[0]
    cv2.circle(img,(x,y),5,[0,0,255],-1)
    
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()    